In [1]:
import sys
print(sys.executable)

/mnt/ffs24/home/lirui15/miniforge3/bin/python


In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdFingerprintGenerator
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D
import seaborn as sns
import matplotlib.pyplot as plt

## Convert .xlsx file to.smi ##

In [3]:
excel_file_path = 'PFAS-SMILES.xlsx'

# Define the path for the output .smi file
smi_file_path = 'SMILES.smi'

# Read the Excel file
df = pd.read_excel(excel_file_path, engine='openpyxl')

# Assume the SMILES strings are in a column named 'SMILES'
# Extract the SMILES column
smiles_strings = df['SMILES']

# Write the SMILES strings to a .smi file
with open(smi_file_path, 'w') as smi_file:
    for smiles in smiles_strings:
        smi_file.write(smiles + '\n')

print(f'SMILES strings have been written to {smi_file_path}')

SMILES strings have been written to SMILES.smi


## Generation of extended connectivity fingerprints ##

In [ ]:
print('reading data')
data = pd.read_csv('PFAS_SMILES.csv',encoding='cp1252')
print('reading data finished')


SMILES = data['SMILES'].to_numpy()

#generate ECFP for all molecules
print('statr generating ECFP')
FP = []
ONS_index = []
for i,sm in enumerate(SMILES):
    if 'N' in sm:#or 'O' in sm or 'S' in sm:
        ONS_index.append(i)
    mol = Chem.MolFromSmiles(sm)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol,6,nBits=1024)
    FP.append(fp.ToBitString())
data['ECFP'] = FP
output_file_path = 'PFAS_SMILES_with_ECFP.csv'
data.to_csv(output_file_path, index=False, encoding='cp1252')
print('finish generating ECFP')

## Exhibition of substructures of the chemicals ##

In [3]:
smiles = 'C(=O)(C(C(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(F)F)(F)F)(F)F)O'

structure_id = 845

mol = Chem.MolFromSmiles(smiles)
bi = {}
fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=6, nBits=1024, bitInfo=bi)

print(bi)

{29: ((3, 4),), 47: ((8, 3),), 59: ((9, 1),), 114: ((2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0)), 207: ((2, 2),), 259: ((5, 5),), 346: ((7, 3),), 374: ((8, 1), (7, 1), (6, 1), (5, 1), (4, 1), (3, 1)), 389: ((27, 1),), 429: ((10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1)), 447: ((4, 3),), 475: ((0, 2),), 544: ((7, 4),), 615: ((4, 5),), 617: ((3, 2),), 619: ((2, 3),), 650: ((1, 0),), 750: ((3, 3),), 790: ((6, 3), (5, 3)), 805: ((2, 1),), 807: ((0, 0), (27, 0)), 845: ((5, 4),), 861: ((6, 4),), 862: ((6, 5),), 886: ((9, 2),), 893: ((1, 1),), 904: ((10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (15, 0), (16, 0), (17, 0), (18, 0), (19, 0), (20, 0), (21, 0), (22, 0), (23, 0), (24, 0), (25, 0), (26, 0)), 916: ((8, 2),), 924: ((4, 4),), 953: ((0, 1),), 972: ((7, 2), (6, 2), (5, 2), (4, 2))}


[17:11:06] DEPRECATION WARNING: please use MorganGenerator


## Visualisation of the substructure of a specific ECFP bit, e.g., ECFP 846 ##

In [2]:
smiles = 'C(=O)(C(C(C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)F)(F)F)(F)F)(F)F)O'
target_bit = 845  # Bit to visualize (e.g., ECFP-845)

# Generate molecule from SMILES
mol = Chem.MolFromSmiles(smiles)
if mol is None:
    raise ValueError("SMILES parsing failed. Please check the input format.")

# Calculate Morgan fingerprint and extract bit info
bit_info = {}
fp = AllChem.GetMorganFingerprintAsBitVect(
    mol, 
    radius=6, 
    nBits=1024, 
    bitInfo=bit_info
)

# Check if the target bit exists
if target_bit in bit_info:
    print(f"Bit {target_bit} corresponds to the following atom environments:")
    print(bit_info[target_bit])  # Output: [(center_atom_idx, radius)]
    
    # Get all atoms involved in the feature
    highlight_atoms = set()
    for (center_atom, radius) in bit_info[target_bit]:
        # Find all atoms within the specified radius
        env_atoms = Chem.FindAtomEnvironmentOfRadiusN(mol, radius, center_atom)
        highlight_atoms.update(env_atoms)
    
    # Draw and save the highlighted structure
    drawer = rdMolDraw2D.MolDraw2DCairo(500, 500)
    drawer.DrawMolecule(mol, highlightAtoms=list(highlight_atoms))
    drawer.FinishDrawing()
    
    # Save image to file
    output_file = f"ecfp_bit_{target_bit}.png"
    with open(output_file, "wb") as f:
        f.write(drawer.GetDrawingText())
    print(f"Image saved to: {output_file}")
    

Bit 845 corresponds to the following atom environments:
((5, 4),)
Image saved to: ecfp_bit_845.png


[17:10:27] DEPRECATION WARNING: please use MorganGenerator


## Separated the bits into multiple columns ##

In [6]:
ECFP_file_path = 'PFAS_SMILES_with_ECFP.csv'
df = pd.read_csv(ECFP_file_path)
ecfp = 'ECFP'

def split_ecfp(row):
    ecfp_str = row[ecfp]
    return pd.Series(list(ecfp_str))

split_columns = df.apply(split_ecfp, axis=1)

In [7]:
split_columns.columns = [f'ECFP_{i+1}' for i in range(split_columns.shape[1])]
df = pd.concat([df, split_columns], axis=1)

In [8]:
output_file_path = 'PFAS_SMILES_with_ECFP_split.csv' 
df.to_csv(output_file_path, index=False)

print(f"Processed data saved to {output_file_path}")

Processed data saved to PFAS_SMILES_with_ECFP_split.csv


## Remove 0-variance bits ##

In [12]:
file_path = 'PFAS_SMILES_with_ECFP_split.csv' 
df = pd.read_csv(file_path)
variance = df.iloc[:, 4:].var()

columns_to_drop = variance[variance == 0].index
df_cleaned = df.drop(columns=columns_to_drop, axis=1)

output_file_path = 'PFAS_ECFP_cleaned1.csv' 
df_cleaned.to_csv(output_file_path, index=False)


## Eliminate highly correlated bits ##

In [9]:
file_path = 'PFAS_ECFP_cleaned1.csv' 
df = pd.read_csv(file_path)

df_subset = df.iloc[:, 4:]
spearman_corr_matrix = df_subset.corr(method='spearman')

output_corr_matrix_path = 'spearman_corr_matrix-ECFP.csv'
spearman_corr_matrix.to_csv(output_corr_matrix_path)

In [ ]:
sns.heatmap(spearman_corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', 
            vmin=-1, vmax=1, center=0, cbar=True, square=True, linewidths=0.5, linecolor='black')
plt.title('Spearman Correlation Heatmap')
plt.show()